In [1]:
# standard library imports
from __future__ import absolute_import, division, print_function

import keras

# standard numerical library imports
import pandas
import numpy as np
import scipy as sp

# energyflow imports
import energyflow as ef
from energyflow.archs import PFN
from energyflow.utils import data_split, remap_pids, to_categorical

import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

Using TensorFlow backend.
/root/anaconda3/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/anaconda3/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/anaconda3/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/root/anaconda3/lib/python3.5/site-packages/tensorflow/python/framework/dtype

In [2]:
jet_part_sub_all_175 = pandas.read_csv("../output_particles_175.csv")
jet_part_sub_all_175["m"]=0.175
jet_part_sub_175 = [y for x,y in jet_part_sub_all_175.groupby(["entry"])]
len(jet_part_sub_175)

10000

In [3]:
jet_part_sub_all_180 = pandas.read_csv("../output_particles_180.csv")
jet_part_sub_all_180["m"]= 0.180
jet_part_sub_180 = [y for x,y in jet_part_sub_all_180.groupby(["entry"])]
len(jet_part_sub_180)

10000

In [4]:
jet_size_175 = []
for i in range(len(jet_part_sub_175)):
    jet_size_175.append(len(jet_part_sub_175[i]))

jet_size_180 = []
for i in range(len(jet_part_sub_180)):
    jet_size_180.append(len(jet_part_sub_180[i]))

In [5]:
np.average(jet_size_175), np.average(jet_size_180)

(5.0748, 5.0911)

In [6]:
np.min(jet_size_175), np.min(jet_size_180)

(1, 1)

In [7]:
np.max(jet_size_175), np.max(jet_size_180)

(13, 13)

In [8]:
max_jet_size = max(np.max(jet_size_175), np.max(jet_size_180))

We want N X 1200 X 4

In [9]:
Y0 = np.zeros((int(len(jet_part_sub_175)/3)))
Y1 = np.ones((int(len(jet_part_sub_180)/3)))
Y  = np.concatenate([Y0,Y1])
Y = to_categorical(Y, num_classes=2)

In [10]:
feature_to_use = ["pT","eta","phi","btag","m"]
num_feature = len(feature_to_use)

In [11]:
X0_all = np.zeros((len(jet_part_sub_175), max_jet_size,num_feature))
X1_all = np.zeros((len(jet_part_sub_180), max_jet_size,num_feature))

In [44]:
X0_all.shape

(10000, 13, 5)

In [12]:
for i in range(len(jet_part_sub_175)):
    jets = np.array(jet_part_sub_175[i][feature_to_use])
    X0_all[i,:len(jets),:] = jets[:max_jet_size]

In [13]:
jet_part_sub_175[3]["btag"]

15    0
16    1
17    1
18    0
Name: btag, dtype: int64

In [14]:
for i in range(len(jet_part_sub_180)):
    jets = np.array(jet_part_sub_180[i][feature_to_use])
    X1_all[i,:len(jets),:] = jets[:max_jet_size]

In [15]:
X0_train = X0_all[:int(len(jet_part_sub_175)/3),:,:]
X1_train = X0_all[:int(len(jet_part_sub_180)/3),:,:]
X0_test = X0_all[int(len(jet_part_sub_175)/2):int(len(2*jet_part_sub_180)/3),:,:]
X1_test = X1_all[int(len(jet_part_sub_180)/2):int(len(2*jet_part_sub_180)/3),:,:]

In [16]:
X0_train.shape, X1_train.shape

((3333, 13, 5), (3333, 13, 5))

In [17]:
X = np.concatenate([X0_train,X1_train])

In [18]:
print(np.sum(X[1,:,0]))

420.3793


In [19]:
# remap_pids(X, pid_i=0)

In [20]:
for i in range(len(X)):
    X[i] = normalize(X[i])

In [21]:
X_train, X_val, Y_train, Y_val = data_split(X, Y, test=0.3, shuffle=True)

In [22]:
print(X_train.shape)
print(Y_train.shape)

print(X_val.shape)
print(Y_val.shape)

(4667, 13, 5)
(4667, 2)
(1999, 13, 5)
(1999, 2)


In [23]:
# network architecture parameters
Phi_sizes = (100,100, 128)
F_sizes = (100,100, 100)

dctr = PFN(input_dim= num_feature, 
           Phi_sizes=Phi_sizes, F_sizes=F_sizes,
           summary=False)

In [24]:
save_label = 'DCTR_top_tagging'

checkpoint = keras.callbacks.ModelCheckpoint(save_label + '.h5', 
                                                monitor='val_loss', 
                                                verbose=2, 
                                                save_best_only=True, 
                                                mode='min')

CSVLogger = keras.callbacks.CSVLogger(save_label + '_loss.csv', append=False)

EarlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                              min_delta=0, 
                                              patience=10, 
                                              verbose=1, 
                                              restore_best_weights=True)

callbacks = [checkpoint, CSVLogger, EarlyStopping]

In [25]:
history = dctr.fit(X_train, Y_train,
                    epochs = 10,
                    batch_size = 1,
                    validation_data = (X_val, Y_val),
                    verbose = 1, 
                    callbacks = callbacks)

Train on 4667 samples, validate on 1999 samples
Epoch 1/10
4667/4667 [==============================] - 27s 6ms/step - loss: 0.7131 - acc: 0.4905 - val_loss: 0.6941 - val_acc: 0.4877

Epoch 00001: val_loss improved from inf to 0.69406, saving model to DCTR_top_tagging.h5
Epoch 2/10
4667/4667 [==============================] - 20s 4ms/step - loss: 0.6936 - acc: 0.5046 - val_loss: 0.6933 - val_acc: 0.4877

Epoch 00002: val_loss improved from 0.69406 to 0.69326, saving model to DCTR_top_tagging.h5
Epoch 3/10
4667/4667 [==============================] - 18s 4ms/step - loss: 0.6936 - acc: 0.4956 - val_loss: 0.6935 - val_acc: 0.4877

Epoch 00003: val_loss did not improve from 0.69326
Epoch 4/10
4667/4667 [==============================] - 18s 4ms/step - loss: 0.6934 - acc: 0.5007 - val_loss: 0.6938 - val_acc: 0.4877

Epoch 00004: val_loss did not improve from 0.69326
Epoch 5/10
4667/4667 [==============================] - 24s 5ms/step - loss: 0.6935 - acc: 0.4965 - val_loss: 0.6933 - val_acc

In [26]:
preds_0 = dctr.predict(X0_test, batch_size=1)
preds_1 = dctr.predict(X1_test, batch_size=1)

In [27]:
preds_0,preds_1

(array([[0.4866167 , 0.5133833 ],
        [0.43823975, 0.5617603 ],
        [0.43075877, 0.5692413 ],
        ...,
        [0.57236284, 0.42763722],
        [0.48088357, 0.51911646],
        [0.4826729 , 0.51732713]], dtype=float32),
 array([[0.5041373 , 0.49586272],
        [0.4710738 , 0.52892613],
        [0.50332314, 0.49667692],
        ...,
        [0.44974113, 0.5502588 ],
        [0.47740132, 0.5225986 ],
        [0.5651662 , 0.43483388]], dtype=float32))

In [28]:
weights_0 = preds_0[:,0]/preds_0[:,1]
weights_1 = preds_1[:,0]/preds_1[:,1]

In [29]:
print(max(weights_0))
print(max(1/weights_0))
print(max(weights_1))
print(max(1/weights_1))

2.5725048
2.7264304
1.9646866
2.3508766


In [30]:
class AddParams2Input(keras.layers.Layer):
    """ Custom layer for tuning with DCTR: 
    Arguments:
    - n_MC_params : (int) - the number of n_MC_params that are in X_dim
    - default_MC_params : (list of floats) - default values for each of the MC parameters
    - trainable_MC_params : (list of booleans) - True for parameters that you want to fit, false for parameters that should be fixed at default value

    Usage: 
    Let X_dim be the input dimension of each particle to a PFN model, and n_MC_params be the number of MC parameters. 
    Defines a Layer that takes in an array of dimension 
    (batch_size, padded_multiplicity, X_dim - n_MC_params)
    This layer appends each particle by the default_MC_params and makes then trainable or non-trainable based on trainable_MC_params
    """
    
    def __init__(self, n_MC_params, default_MC_params, trainable_MC_params):
        super(AddParams2Input, self).__init__()
        # Definitions
        self.n_MC_params = n_MC_params
        self.MC_params = default_MC_params
        self.trainable_MC_params = trainable_MC_params

    
    def build(self, input_shape):
        # Convert input MC parameters to weights and make then trainable or non-trainable
        for i in range(self.n_MC_params):
            self.MC_params[i] = self.add_weight(name='MC_param_{}'.format(i), 
                                                shape=(1, 1),
                                                initializer=keras.initializers.Constant(self.MC_params[i]),
                                                trainable=self.trainable_MC_params[i])
            
        self.MC_params = keras.backend.tf.concat(self.MC_params, axis = -1)
        super(AddParams2Input, self).build(input_shape)
    
    def call(self, input):
        # Add MC params to each input particle (but not to the padded rows)
        concat_input_and_params = keras.backend.tf.where(keras.backend.abs(input[...,0])>0,
                                                         self.MC_params*keras.backend.ones_like(input[...,0:self.n_MC_params]),
                                                         keras.backend.zeros_like(input[...,0:self.n_MC_params]))
        return keras.backend.concatenate([input, concat_input_and_params], -1)
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1]+self.n_MC_params)

In [31]:
def get_DCTR_fit_model(DCTR_model, 
                       X_dim, 
                       n_MC_params, 
                       default_MC_params,
                       trainable_MC_params):
    """ 
    Get a DCTR model that trains on the input MC parameters
    
    Arguments:
    - DCTR_model : a PFN model that has been trained on a to continuously interpolate over the input MC dimensions
    - X_dim : (int) - the dimension of the input expected by DCTR_model
    - n_MC_params : (int) - the number of n_MC_params that are in X_dim
    - default_MC_params : (list of floats) - default values for each of the MC parameters
    - trainable_MC_params : (list of booleans) - True for parameters that you want to fit, false for parameters that should be fixed at default value

    Returns:
    - DCTR_fit_model: a compiled model that gradient descends only on the trainable MC parameters
    """
    
    # Do sanity checks on inputs
    assert X_dim >=n_MC_params, "X_dim must be larger than n_MC_params. X_dim includes the dimensionality of the 4-vector + number of MC parameters"
    assert n_MC_params == len(default_MC_params), "Dimension mismatch between n_MC_params and number of default MC parameters given. len(default_MC_params) must equal n_MC_params"
    assert n_MC_params == len(trainable_MC_params), "Dimension mismatch between n_MC_params and trainable_MC_params. len(trainable_MC_params) must equal n_MC_params."
    assert np.any(trainable_MC_params), "All parameters are set to non-trainable."
    
    # Define input to DCTR_fit_model
    non_param_input = keras.layers.Input((None, X_dim - n_MC_params))

    # Construct layer that adds trainable and non-trainable parameters to the input
    add_params_layer = AddParams2Input(n_MC_params, default_MC_params, trainable_MC_params)
    time_dist     = keras.layers.TimeDistributed(add_params_layer, name='tdist')(non_param_input)     

    # Set all weights in DCTR_model to non-trainable
    for layer in DCTR_model.model.layers:
        layer.trainable = False
        
    # get the graph and the weights from the DCTR_model
    output = DCTR_model.model(inputs = time_dist)

    # Define full model
    DCTR_fit_model = fitmodel = keras.models.Model(inputs = non_param_input, outputs = output)
    
    optimizer = keras.optimizers.Adam(lr=1e-4)
    
    # Compile with loss function
    DCTR_fit_model.compile(optimizer=optimizer, loss='categorical_crossentropy')
    
    return DCTR_fit_model

In [32]:
dctr_fit_model = get_DCTR_fit_model(dctr, 
                       X_dim =5, 
                       n_MC_params = 1, 
                       default_MC_params   = [0.175], # default params for [alpha_s, aLund, StoUD]
                       trainable_MC_params = [True]) # Only train aLund

dctr_fit_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 4)           0         
_________________________________________________________________
tdist (TimeDistributed)      (None, None, 5)           1         
_________________________________________________________________
model_1 (Model)              (None, 2)                 56930     
Total params: 56,931
Trainable params: 1
Non-trainable params: 56,930
_________________________________________________________________


In [33]:
def set_MC_params(dctr_fit_model, MC_params):
    top_mass = MC_params
    weights = np.array([[top_mass]],dtype=np.float32)
    dctr_fit_model.layers[1].set_weights(weights)

In [34]:
dctr_fit_model.layers[1].get_weights()[0].shape

(1, 1)

In [35]:
X.shape, Y.shape

((6666, 13, 5), (6666, 2))

In [36]:
X0_fit = X0_all[int(2*len(jet_part_sub_175)/3):9999,:,:]
X1_fit = X1_all[int(2*len(jet_part_sub_180)/3):9999,:,:]
X_fit = np.concatenate([X0_fit,X1_fit])[:,:,:4]

In [37]:
Y_fit = Y[:len(X_fit)]

In [38]:
X_fit.shape,Y_fit.shape

((6666, 13, 4), (6666, 2))

In [39]:
X_fit, _, Y_fit, _ = data_split(X_fit, Y_fit, test=0, shuffle=True)

In [40]:
def get_loss(X, Y, dctr_fit_model, MC_params, batch_size = 1000):
    set_MC_params(dctr_fit_model, MC_params)
    return dctr_fit_model.evaluate(x=X, y = Y, batch_size=batch_size)

In [41]:
dctr_fit_model.layers[1].get_weights()

[array([[0.175]], dtype=float32)]

In [42]:
top_mass_loss = np.array([(top_mass, get_loss(X_fit, Y_fit, dctr_fit_model, [top_mass])) for top_mass in np.linspace(0.170,0.190, 31)])

6666/6666 [==============================] - 0s 71us/step


In [43]:
print(top_mass_loss)

[[0.17       0.69458045]
 [0.17066667 0.69458099]
 [0.17133333 0.69458152]
 [0.172      0.69458199]
 [0.17266667 0.69458256]
 [0.17333333 0.69458304]
 [0.174      0.69458355]
 [0.17466667 0.69458412]
 [0.17533333 0.69458455]
 [0.176      0.69458502]
 [0.17666667 0.69458555]
 [0.17733333 0.69458608]
 [0.178      0.69458663]
 [0.17866667 0.69458716]
 [0.17933333 0.69458772]
 [0.18       0.69458822]
 [0.18066667 0.69458876]
 [0.18133333 0.69458923]
 [0.182      0.69458977]
 [0.18266667 0.69459031]
 [0.18333333 0.69459086]
 [0.184      0.69459117]
 [0.18466667 0.69459169]
 [0.18533333 0.69459208]
 [0.186      0.69459243]
 [0.18666667 0.69459284]
 [0.18733333 0.69459322]
 [0.188      0.69459364]
 [0.18866667 0.69459394]
 [0.18933333 0.69459427]
 [0.19       0.69459471]]
